In [1]:
import numpy as np
import pandas as pd
import apogee_tools as ap
import emcee

/home/jess/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def makeModel(**kwargs):

    params = kwargs.get('params')
    fiber  = kwargs.get('fiber', 40)
    plot   = kwargs.get('plot', False)
    res    = kwargs.get('res', '300k')
    grid   = kwargs.get('grid', 'phoenix').lower()

    mdl_name = r'Teff = {}, logg = {}, Fe/H = {}, vsini = {}, rv = {}, $\alpha$ = {}'.format(params['teff'], params['logg'], params['z'], params['vsini'], params['rv'], params['alpha'])
    labels = [params['teff'], params['logg'], params['z']]

    #Interpolate model grids at give teff, logg, fe/h
    interp_sp = ap.interpolateGrid(labels=labels, res=res, grid=grid)
    interp_sp.flux = interp_sp.flux/max(interp_sp.flux)

    #Apply radial velocity
    rv_sp   = ap.spec_tools.rvShiftSpec(interp_sp, rv=params['rv'])

    #Apply rotational velocity broadening
    rot_sp  = ap.applyVsini(rv_sp, vsini=params['vsini'])

    #Apply telluric spectrum
    tell_sp = ap.applyTelluric(rot_sp, alpha=params['alpha'])

    #Apply APOGEE LSF function
    lsf_sp  = ap.convolveLsf(tell_sp, fiber=fiber)

    synth_model = ap.Spectrum(wave=lsf_sp.wave, flux=lsf_sp.flux, name=mdl_name)

    return synth_model

In [ ]:
def lnprior(model):

    teff, logg,  fe_h, rv, vsini = model.theta  #parameters should be arranged in this manner
    
    if  (prior["teff"][0] < teff < 4000) and (3.0 < logg  < 6.0) \
    and (-2.0 < fe_h < 2.0) and (-50.0< rv < 50.0) \
    and (0 < vsini < 20):
        return 0.0
    return -np.inf

In [ ]:
def run_MCMC(**kwargs):
    """
    MCMC run using EMCEE
    params: model (must have a chi-square and a list of parameters)
    
    optional params: 
    first guess: guess
    nwalkers: number of walkers
    nsamples: number of runs default: 10000
    
    """
    #model to use for initial guess if not given
    guess_model = kwargs.get('guess_model', ToyModel())
    guess = kwargs.get('guess', guess_model.theta)
    params = np.array(guess)
    
    #noise=kwargs.get('noise', spectrum.noise.value)
    ndim=len(model.theta)
    nwalkers=kwargs.get('nwalkers', 10)
    nsamples=kwargs.get('nsamples', 1000)
    
    #make the first guess the same for all walkers with some random number added
    p0=[guess for n in range(0,nwalkers)]
    for n in range(1, nwalkers):
        p0[n]=[p+ np.random.random(1)[0]*0.0001*p for p in params]
    
    print ("walkers {} initial guess for each walker {} samples {}".format(nwalkers, p0, nsamples))

    
    #make this a numpy array
    p0= np.array(p0)
        
    #emcee sampler object
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnlike)
    
    #run the MCMC 
    ps, lnps, rstate= sampler.run_mcmc(p0,nsamples)
    
    #get the posteriors
    samples = sampler.chain.reshape((-1, ndim))
    
    #get mean parameters
    pmean=[np.nanmean((samples.T)[i]) for i in range(0, ndim)]
    #get standard deviations
    pstd=[np.nanstd((samples.T)[i]) for i in range(0, ndim)]
    
    #pf= [np.mean(((samples.T)[i])[-10:]) for i in range(0, ndim)]
    
    #visualization 
    param_names=['teff', 'logg',  'fe_h', 'rv', 'vsini']
    #std keys
    param_er_names=['teff_er', 'logg_er',  'fe_h_er', 'rv_er', 'vsini_er']
    
      
    if kwargs.get('show_corner', True):
        plt.figure()
        fig = corner.corner(samples, labels=param_names)
        plt.show()
        
        plt.figure()
        fig, ax = plt.subplots(ndim, sharex=True, figsize=(12, 6))
        for i in range(ndim):
            ax[i].plot(sampler.chain[:, :, i].T, '-k', alpha=0.2)
        fig.show()
    
    return samples, dict(zip(param_names, pmean)), dict(zip(param_er_names, pstd))

In [6]:
k = np.array([1,2,3])
type(k) == 'np.ndarray'

False